# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf



In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [5]:
# Carregar a base de dados
df = pd.read_csv("previsao_de_renda.csv")

# Remover colunas desnecessárias
df = df.drop(columns=["Unnamed: 0", "data_ref", "id_cliente"])

# Remover valores ausentes
df = df.dropna()

# Converter variáveis categóricas em dummies
df_dummies = pd.get_dummies(df, drop_first=True)

# Separar X (variáveis explicativas) e y (variável alvo)
X = df_dummies.drop(columns="renda")
y = df_dummies["renda"]

# Dividir em treino (75%) e teste (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Verificar shapes das bases
print("Shape de X_train:", X_train.shape)
print("Shape de X_test:", X_test.shape)
print("Shape de y_train:", y_train.shape)
print("Shape de y_test:", y_test.shape)

Shape de X_train: (9320, 24)
Shape de X_test: (3107, 24)
Shape de y_train: (9320,)
Shape de y_test: (3107,)


In [6]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Lista de alphas para testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Dicionário para armazenar os R²
resultados_ridge = {}

# Treinar e avaliar para cada alpha
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    resultados_ridge[alpha] = r2
    print(f"Alpha = {alpha}: R² na base de teste = {r2:.4f}")

# Identificar o melhor alpha
melhor_alpha_ridge = max(resultados_ridge, key=resultados_ridge.get)
print(f"\nMelhor modelo Ridge: alpha = {melhor_alpha_ridge} com R² = {resultados_ridge[melhor_alpha_ridge]:.4f}")

Alpha = 0: R² na base de teste = 0.2980
Alpha = 0.001: R² na base de teste = 0.2980
Alpha = 0.005: R² na base de teste = 0.2980
Alpha = 0.01: R² na base de teste = 0.2980
Alpha = 0.05: R² na base de teste = 0.2980
Alpha = 0.1: R² na base de teste = 0.2980

Melhor modelo Ridge: alpha = 0.1 com R² = 0.2980


In [7]:
# Lasso
from sklearn.linear_model import Lasso

# Lista de alphas para testar
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Dicionário para armazenar os R²
resultados_lasso = {}

# Treinar e avaliar para cada alpha
for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)  # aumenta max_iter se necessário
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    resultados_lasso[alpha] = r2
    print(f"Alpha = {alpha}: R² na base de teste = {r2:.4f}")

# Identificar o melhor alpha
melhor_alpha_lasso = max(resultados_lasso, key=resultados_lasso.get)
print(f"\nMelhor modelo LASSO: alpha = {melhor_alpha_lasso} com R² = {resultados_lasso[melhor_alpha_lasso]:.4f}")



/home/luiza/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/home/luiza/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/luiza/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model 

Alpha = 0: R² na base de teste = 0.2980
Alpha = 0.001: R² na base de teste = 0.2980
Alpha = 0.005: R² na base de teste = 0.2980
Alpha = 0.01: R² na base de teste = 0.2980
Alpha = 0.05: R² na base de teste = 0.2980
Alpha = 0.1: R² na base de teste = 0.2980

Melhor modelo LASSO: alpha = 0.1 com R² = 0.2980


In [11]:
#stepwise
import statsmodels.api as sm
import numpy as np

# Converter todos os dados para float 
X_train = X_train.astype(float)
X_test = X_test.astype(float)
y_train = y_train.astype(float)
y_test = y_test.astype(float)


def stepwise_selection(X, y, 
                       initial_features=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    """Perform a forward-backward feature selection based on p-value from statsmodels.api.OLS"""
    included = list(initial_features)
    while True:
        changed = False

        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_col in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_col]]))).fit()
            new_pval[new_col] = model.pvalues[new_col]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f"Adicionada: {best_feature} com p-valor {best_pval:.6f}")

        # Backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # maior p-valor
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f"Removida: {worst_feature} com p-valor {worst_pval:.6f}")

        if not changed:
            break

    return included

# Executar stepwise na base de treinamento
selected_features = stepwise_selection(X_train, y_train)

# Ajustar modelo final com as variáveis selecionadas
X_train_sel = sm.add_constant(X_train[selected_features])
X_test_sel = sm.add_constant(X_test[selected_features])
modelo_stepwise = sm.OLS(y_train, X_train_sel).fit()

# Avaliar R² na base de teste
y_pred_stepwise = modelo_stepwise.predict(X_test_sel)
r2_stepwise = r2_score(y_test, y_pred_stepwise)

print(f"\nR² na base de teste (Stepwise): {r2_stepwise:.4f}")


Adicionada: tempo_emprego com p-valor 0.000000
Adicionada: sexo_M com p-valor 0.000000
Adicionada: tipo_renda_Empresário com p-valor 0.000004
Adicionada: idade com p-valor 0.000024
Adicionada: educacao_Superior completo com p-valor 0.000804

R² na base de teste (Stepwise): 0.2969


In [12]:
# Coeficientes do modelo Ridge
ridge_best = Ridge(alpha=0.1)
ridge_best.fit(X_train, y_train)
coef_ridge = pd.Series(ridge_best.coef_, index=X_train.columns)

# Coeficientes do modelo Lasso
lasso_best = Lasso(alpha=0.1)
lasso_best.fit(X_train, y_train)
coef_lasso = pd.Series(lasso_best.coef_, index=X_train.columns)

# Coeficientes do modelo Stepwise
coef_stepwise = pd.Series(modelo_stepwise.params[1:], index=selected_features)  # ignora o intercepto
coef_stepwise_full = pd.Series(0, index=X_train.columns)
coef_stepwise_full.update(coef_stepwise)

# Juntar tudo em um DataFrame para comparar
coef_df = pd.DataFrame({
    'Ridge': coef_ridge,
    'Lasso': coef_lasso,
    'Stepwise': coef_stepwise_full
})

# Exibir coeficientes diferentes de zero em pelo menos um modelo
coef_df_filtrado = coef_df[(coef_df != 0).any(axis=1)]
print(coef_df_filtrado.sort_values(by='Ridge', ascending=False).round(3))

                                  Ridge     Lasso  Stepwise
sexo_M                         6212.241  6211.542  6191.236
qt_pessoas_residencia          1456.506  1360.068     0.000
estado_civil_Separado          1334.412  1236.266     0.000
educacao_Superior completo     1300.310  1256.156   582.120
estado_civil_Viúvo             1286.280  1185.255     0.000
estado_civil_Solteiro          1055.037   959.150     0.000
tipo_renda_Empresário           887.794   887.469   840.551
educacao_Secundário             760.247   716.750     0.000
educacao_Pós graduação          584.063   485.505     0.000
tempo_emprego                   571.107   571.082   570.396
posse_de_imovel                 361.232   361.180     0.000
educacao_Superior incompleto    337.566   292.402     0.000
tipo_renda_Servidor público     127.816   126.944     0.000
idade                            40.065    40.070    42.890
posse_de_veiculo               -133.340  -132.724     0.000
tipo_residencia_Governamental  -235.956 

/home/luiza/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.780e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(
/tmp/ipykernel_210473/3094004925.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[  42.89001763  570.39562878 6191.23569438  840.55134306  582.12025377]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  coef_stepwise_full.update(coef_stepwise)


Os modelos Ridge e Lasso apresentaram melhor desempenho em termos de 𝑅2, com vantagem para Lasso na redução de variáveis com baixa relevância. 

In [14]:
# Copiar bases para não alterar original
X_train_mod = X_train.copy()
X_test_mod = X_test.copy()

# Transformações
X_train_mod['log_idade'] = np.log1p(X_train_mod['idade'])
X_test_mod['log_idade'] = np.log1p(X_test_mod['idade'])

X_train_mod['log_tempo_emprego'] = np.log1p(X_train_mod['tempo_emprego'])
X_test_mod['log_tempo_emprego'] = np.log1p(X_test_mod['tempo_emprego'])

X_train_mod['idade_emprego'] = X_train_mod['idade'] * X_train_mod['tempo_emprego']
X_test_mod['idade_emprego'] = X_test_mod['idade'] * X_test_mod['tempo_emprego']

X_train_mod['tempo_emprego2'] = X_train_mod['tempo_emprego'] ** 2
X_test_mod['tempo_emprego2'] = X_test_mod['tempo_emprego'] ** 2

# Rodar Ridge novamente
ridge_mod = Ridge(alpha=0.1)
ridge_mod.fit(X_train_mod, y_train)
y_pred_mod = ridge_mod.predict(X_test_mod)
r2_mod = r2_score(y_test, y_pred_mod)

print(f"R² com transformações = {r2_mod:.4f}")


R² com transformações = 0.2983


In [16]:
# Ajustar a árvore
tree = DecisionTreeRegressor(max_depth=5, random_state=42)
tree.fit(X_train, y_train)

# R²
y_pred_tree = tree.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)

print(f"R² da árvore de regressão (max_depth=5): {r2_tree:.4f}")


R² da árvore de regressão (max_depth=5): 0.3610
